<a href="https://colab.research.google.com/github/Alinehbg/CRP/blob/Working_Directory/Extract%20Names/Extract_Wiki_Data_names_try_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install requests
import requests

####testing requests with get and post

In [2]:
#USING GET REQUEST
test_query = """SELECT DISTINCT ?item ?itemLabel WHERE {
    ?item rdfs:label "Baker"@en;
          wdt:P31 wd:Q101352.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}"""

#encoded_query = requests.utils.quote(test_query) I dont think we need this
url = f'https://query.wikidata.org/sparql?query={test_query}&format=json'
response = requests.get(url)

data = response.json()
data

{'head': {'vars': ['item', 'itemLabel']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q2658150'},
    'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Baker'}}]}}

In [3]:
#USING POST REQUEST
data = {'query': test_query}
response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
data = response.json()
data

# dont think I need to specify the content-type as application/x-www-form-urlencoded while sending post request.
# headers = {'content-type': 'application/x-www-form-urlencoded'}
# response = requests.post('https://query.wikidata.org/sparql?format=json', data=data,headers=headers)


{'head': {'vars': ['item', 'itemLabel']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q2658150'},
    'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Baker'}}]}}

#### All the queries

In [ ]:
#QUERIES
#find female and male politicians
male_politicians_query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item wdt:P106 wd:Q82955; 
            wdt:P21 wd:Q6581097.
    }
    ORDER BY RAND()
  }
  FILTER REGEX(?itemLabel, "^[A-Za-z]+ [A-Za-z]+$")
}
LIMIT 10
'''
#male Q6581097

female_politicians_query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item wdt:P106 wd:Q82955; 
            wdt:P21 wd:Q6581072.
    }
    ORDER BY RAND()
  }
  FILTER REGEX(?itemLabel, "^[A-Za-z]+ [A-Za-z]+$")
}
LIMIT 10
'''
#female Q6581072

In [ ]:
#Queries

SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P460 ?statement0.
      ?statement0 (ps:P460/(wdt:P279*)) wd:Q21055025.
    }
    LIMIT 100
  }
}

#### functions 

In [4]:
def get_itemlabel(query): 
  name_list = []
  for x in query["results"]["bindings"]: 
    name_list.append(x["itemLabel"]["value"])
  return name_list

def find_fname_id(name): 
  find_fn_id_query = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query}
  response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  res = response.json()
  try:
      id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
      #print(f"Cannot find first name id for {name}")  
      id = []  
  return id

def find_lname_id(name): 
  find_fn_id_query = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q101352.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query}
  response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  res = response.json()
  try:
      id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
      #print(f"Cannot find last name id for {name}")  
      id = []  
  return id

def find_similar_from_id(id): 
  #find similar names from the ID just found for the target name
  id = id
  find_similar_query = f"""
  SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    {{
      SELECT DISTINCT ?item WHERE {{
        ?item p:P460 ?statement0.
        ?statement0 (ps:P460/(wdt:P279*)) wd:{id}.
      }}
    }}
  }}
  """
  data = {'query': find_similar_query}
  response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  res = response.json()
  similar_name_list = get_itemlabel(res)
  return similar_name_list

def similar_full_names(name): 
  f_name, l_name = name.split(' ')

  f_name_id = find_fname_id(f_name)
  l_name_id = find_lname_id(l_name)

  if f_name_id == []: 
    fname_similar = []
    lname_similar = find_similar_from_id(l_name_id)
    fname_list = []
    lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]
  elif l_name_id == []: 
    lname_similar = []
    print(f_name_id)
    fname_similar = find_similar_from_id(f_name_id)
    lname_list = []
    fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
  else: 
    fname_similar = find_similar_from_id(f_name_id)
    lname_similar = find_similar_from_id(l_name_id)

    fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
    lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]
  
  similar_full_names = fname_list + lname_list
  pair = {name: similar_full_names}
  return pair

def Merge(dict1, dict2):
  return(dict2.update(dict1))

In [ ]:
name = 'Joshua Baker'
print(find_fname_id('Joshua'))
print(find_similar_from_id('Q3284319'))
print(similar_full_names(name))
name1 = 'Ernst Albrecht'
print(similar_full_names(name1))

Q3284319
['Joshuah', 'Josh', 'Jozue', 'Josua', 'Joosua', 'Yosua', 'Josuah', 'Giosuè', 'Yehoshua']
{'Joshua Baker': ['Joshuah Baker', 'Josh Baker', 'Jozue Baker', 'Josua Baker', 'Joosua Baker', 'Yosua Baker', 'Josuah Baker', 'Giosuè Baker', 'Yehoshua Baker', 'Joshua Bäcker', 'Joshua Bakker', 'Joshua Becker', 'Joshua Boulanger', 'Joshua Boulenger']}
{'Ernst Albrecht': ['Ernestan Albrecht', 'Ernest Albrecht', 'Ernesto Albrecht', 'Arnošt Albrecht', 'Ernő Albrecht', 'Q111549586 Albrecht', 'Ernst Abel', 'Ernst Albert', 'Ernst Alberto', 'Ernst Avraham', 'Ernst Alberts', 'Ernst Albertus', 'Ernst Albracht', 'Ernst Adelbrecht', 'Ernst Albercht', 'Ernst Allbrecht']}


####requests and the rest

In [ ]:
data = {'query': male_politicians_query}
response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
data = response.json()

male_politicans = [ x["itemLabel"]["value"] for x in data['results']['bindings']]

In [ ]:
data = {'query': female_politicians_query}
response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
data = response.json()

female_politicans = [ x["itemLabel"]["value"] for x in data['results']['bindings']]

In [ ]:
male_politicans

['Volker Kauder',
 'Ernst Uhrlau',
 'Carlo Schmid',
 'Paul Singer',
 'Johannes Janssen',
 'Benedict Waldeck',
 'Hans Biebow',
 'Julius Curtius',
 'Max Warburg',
 'Arpad Wigand']

In [ ]:
for name in male_politicans: 
  print(similar_full_names(name))

{'Jerzy Hausner': ['Jyrki Hausner', 'Georg Hausner', 'György Hausner', 'Juraj Hausner', 'George Hausner', 'Georgi Hausner', 'Göran Hausner', 'Georges Hausner', 'Jürg Hausner', 'Jurgens Hausner', 'Jure Hausner', 'Jord Hausner', 'Jordi Hausner', 'Georgi Hausner', 'Gheorghe Hausner', 'Jorge Hausner', 'Jiří Hausner', 'Juro Hausner', 'Jurij Hausner', 'Georgios Hausner', 'Giorgio Hausner', 'Gorch Hausner', 'Yuri Hausner', 'Georgius Hausner', 'Jurgen Hausner', 'Juri Hausner', 'Xurde Hausner', 'Giorgi Hausner', 'Yrjö Hausner', 'Jörg Hausner', 'Jerguš Hausner', 'Joris Hausner', 'Jörn Hausner', 'Jur Hausner', 'Corc Hausner', 'Đuro Hausner', 'Jorn Hausner', 'Jyri Hausner', 'Juerg Hausner', 'Yegor Hausner', 'Jüri Hausner', 'Jörgen Hausner', 'Jockel Hausner', 'Đorđe Hausner', 'Georgiy Hausner', 'Jørgen Hausner', 'Georgy Hausner', 'Georgi Hausner', 'Jürgen Hausner', 'Gevorg Hausner', 'Jørn Hausner', 'Jurgis Hausner', 'Đurađ Hausner', 'Georgije Hausner', 'Siôr Hausner', 'Xurxo Hausner', 'Chorche Haus

JSONDecodeError: ignored

In [ ]:
print(similar_full_names('Ivo Sanader'))

Q686532
{'Ivo Sanader': ['Juan Sanader', 'Ivan Sanader', 'Ion Sanader', 'Gian Sanader', 'Giovanni Sanader', 'Hans Sanader', 'Jógvan Sanader', 'Johnny Sanader', 'Johannes Sanader', 'Ieuan Sanader', 'John Sanader', 'Honza Sanader', 'Jean Sanader', 'Yves Sanader', 'Siôn Sanader', 'Yannis Sanader', 'Johan Sanader', 'Jan Sanader', 'Johann Sanader', 'Jon Sanader', 'Evan Sanader', 'Joan Sanader', 'Ioannes Sanader', 'Ioann Sanader', 'Yvon Sanader', 'Yvan Sanader', 'Hannes Sanader', 'Joannes Sanader', 'João Sanader', 'Xoán Sanader', 'Ian Sanader', 'Ivashka Sanader', 'Gianni Sanader', 'Sean Sanader', 'Ioan Sanader', 'Ivan Sanader', 'Jehan Sanader', 'Iwan Sanader', 'Jaan Sanader', 'Yanka Sanader', 'Jón Sanader', 'Erwan Sanader', 'Jóhan Sanader', 'Janka Sanader', 'Hanns Sanader', 'Iwo Sanader', 'Giannis Sanader', 'Ivan Sanader', 'Ioannis Sanader', 'Jóhann Sanader', 'Youenn Sanader', 'Yanni Sanader', 'János Sanader', 'Yann Sanader', 'Jóhannes Sanader', 'Erwann Sanader', 'Seán Sanader', 'Ivano Sanad

In [6]:
name_list = similar_full_names('Moustafa Abdelal')

In [8]:
print(name_list)

{'Moustafa Abdelal': ['Mustafa Abdelal', 'Mostafa Abdelal', 'Mustapha Abdelal', 'Moustapha Abdelal', 'Musztafa Abdelal', 'Mostapha Abdelal']}


In [38]:
i = 0
target_name = 'Moustafa Abdelal'
for i in name_list:
  compare_name = name_list[i]


In [39]:
target_name = 'Moustafa Abdelal'
name_list1 = name_list.get(target_name)
matching = []

for i in name_list1:
    matching.append(1)
matching

                  

[1, 1, 1, 1, 1, 1]

In [40]:
print( compare_name)
print(matching)

['Mustafa Abdelal', 'Mostafa Abdelal', 'Mustapha Abdelal', 'Moustapha Abdelal', 'Musztafa Abdelal', 'Mostapha Abdelal']
[1, 1, 1, 1, 1, 1]


Creating dictionary 

In [ ]:
# {target_name: key
#  compare_name: value (for loop) 
#  same: 1}

In [41]:
# import pandas library
import pandas as pd
  
# dictionary with list object in values
details = {
    'Name' : target_name,
    'Variation Name' : compare_name,
    'Match or no match' : matching

}
  
# creating a Dataframe object 
df = pd.DataFrame(details)
  
df

,Name,Variation Name,Match or no match
0,Moustafa Abdelal,Mustafa Abdelal,1
1,Moustafa Abdelal,Mostafa Abdelal,1
2,Moustafa Abdelal,Mustapha Abdelal,1
3,Moustafa Abdelal,Moustapha Abdelal,1
4,Moustafa Abdelal,Musztafa Abdelal,1
5,Moustafa Abdelal,Mostapha Abdelal,1
